На основе датасета из ноутбука проведите анализ данных и постройте зависимости частоты и среднего убытка от какого-либо параметра (за исключением уже выполненных в ноутбуке).

In [1]:
# Загрузка библиотек

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Загрузка набора данных в pandas DataFrame

df = pd.read_csv('freMPL-R.csv', low_memory=False)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343080 entries, 0 to 343079
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           343080 non-null  float64
 1   LicAge             343080 non-null  int64  
 2   RecordBeg          343080 non-null  object 
 3   RecordEnd          181115 non-null  object 
 4   VehAge             177880 non-null  object 
 5   Gender             343080 non-null  object 
 6   MariStat           343080 non-null  object 
 7   SocioCateg         343080 non-null  object 
 8   VehUsage           343080 non-null  object 
 9   DrivAge            343080 non-null  int64  
 10  HasKmLimit         343080 non-null  int64  
 11  BonusMalus         343080 non-null  int64  
 12  VehBody            145780 non-null  object 
 13  VehPrice           145780 non-null  object 
 14  VehEngine          145780 non-null  object 
 15  VehEnergy          145780 non-null  object 
 16  Ve

In [4]:
df.head()

,Exposure,LicAge,RecordBeg,RecordEnd,VehAge,Gender,MariStat,SocioCateg,VehUsage,DrivAge,...,ClaimInd,Dataset,DeducType,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea
0,0.583,366,2004-06-01,NaN,2,Female,Other,CSP1,Professional,55,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.200,187,2004-10-19,NaN,0,Male,Alone,CSP55,Private+trip to office,34,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.083,169,2004-07-16,2004-08-16,1,Female,Other,CSP1,Professional,33,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.375,170,2004-08-16,NaN,1,Female,Other,CSP1,Professional,34,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.500,224,2004-01-01,2004-07-01,3,Male,Other,CSP47,Professional,53,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Смотрим, какие факторы пропущены в каждом из датасетов

dct = {}
for i in range(1,11):
    _x = df.loc[df.Dataset == i].notnull().sum()
    dct[i] = list(_x[_x == 0].index)

print('Dataset  Missing Variables')
for x in range(1,11):
    print(x,'\t',dct[x])

Dataset  Missing Variables
1 	 ['DeducType', 'ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
2 	 ['DeducType', 'ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
3 	 ['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
4 	 ['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
5 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
6 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
7 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
8 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass

In [6]:
# Объединяем наборы данных 5-9, удаляем пустые столбцы, удаляем дубликаты

df59 = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df59 = df59.drop(['Dataset'], axis=1)
df59 = df59.dropna(axis=1, how='all')
print('With duplicates\t\t', len(df59))
df59 = df59.drop_duplicates()
print('Without duplicates\t', len(df59))

With duplicates		 165200
Without duplicates	 115155


In [8]:
df = df59
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115155 entries, 145780 to 310979
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           115155 non-null  float64
 1   LicAge             115155 non-null  int64  
 2   RecordBeg          115155 non-null  object 
 3   RecordEnd          59455 non-null   object 
 4   Gender             115155 non-null  object 
 5   MariStat           115155 non-null  object 
 6   SocioCateg         115155 non-null  object 
 7   VehUsage           115155 non-null  object 
 8   DrivAge            115155 non-null  int64  
 9   HasKmLimit         115155 non-null  int64  
 10  BonusMalus         115155 non-null  int64  
 11  ClaimAmount        115155 non-null  float64
 12  ClaimInd           115155 non-null  int64  
 13  ClaimNbResp        115155 non-null  float64
 14  ClaimNbNonResp     115155 non-null  float64
 15  ClaimNbParking     115155 non-null  float64
 1

In [9]:
df['PolicyCount'] = 1
df['ClaimCount'] = df['ClaimAmount'] > 0
df['NoClaimCount'] = df.PolicyCount - df.ClaimCount
df[['PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].head()

,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
145780,1,0.0,False,1
145781,1,0.0,False,1
145782,1,0.0,False,1
145783,1,0.0,False,1
145784,1,0.0,False,1


In [11]:
df_group_bm = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['BonusMalus']).sum()
df_group_bm = df_group_bm.reset_index()

In [12]:
df_group_bm.head()

,BonusMalus,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
0,50,30659.216,67994,1.112703e+07,5831,62163
1,51,953.531,2218,3.017447e+05,184,2034
2,52,386.436,915,2.110474e+05,83,832
3,53,240.039,551,1.251411e+05,64,487
4,54,1030.683,2413,3.602272e+05,225,2188


In [13]:
fig = px.bar(df_group_bm, x='BonusMalus', y='PolicyCount', title='Распределение числа полисов в зависимости от бонус-малуса')
fig.show()

Судя по всему, 50 - начальный показатель бонус-малуса, поэтому он сильно преобладает

In [14]:
fig = go.Figure(data=[go.Bar(name='No Claims', x=df_group_bm.BonusMalus, y=df_group_bm.NoClaimCount),
                      go.Bar(name='With Claims', x=df_group_bm.BonusMalus, y=df_group_bm.ClaimCount)])
fig.update_layout(barmode='stack', xaxis_title='BonusMalus', yaxis_title='PolicyCount',title='Распределение числа полисов по бонус-малусу с убытками и без')
fig.show()

In [15]:
df_group_bm['Freq'] = df_group_bm.ClaimCount / df_group_bm.Exposure
df_group_bm['AvgClaim'] = df_group_bm.ClaimAmount / df_group_bm.ClaimCount

In [16]:
df_group_bm.head()

,BonusMalus,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,50,30659.216,67994,1.112703e+07,5831,62163,0.190188,1908.253563
1,51,953.531,2218,3.017447e+05,184,2034,0.192967,1639.916702
2,52,386.436,915,2.110474e+05,83,832,0.214783,2542.739242
3,53,240.039,551,1.251411e+05,64,487,0.266623,1955.329362
4,54,1030.683,2413,3.602272e+05,225,2188,0.218302,1601.009934


In [17]:
fig = px.bar(df_group_bm, x='BonusMalus', y='Freq', title='Зависимость частоты убытков от бонус-малуса')
fig.show()

частота убытков растет с увеличением бонус-малуса, но вряд ли эти процессы связаны - скорее убытки становятся чаще т.к. выборка уменьшается с увеличением БМ

In [18]:
fig = px.bar(df_group_bm, x='BonusMalus', y='AvgClaim', title='Зависимость среднего убытка от БМ')
fig.show()

Аномалией выглядит БМ 98, но при том, что таких полисов всего 11, то это скорее выброс. в остальном - примерно похожий размер среднего убытка с повышением для более редких категорий